<a href="https://colab.research.google.com/github/Lakshmi-krishna-vr/Computational_Optimization_/blob/main/Micro_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the optimization library

In [1]:
# Cell 1: Install the optimization library
!pip install pulp

## Import Libraries & Define Raw Data

In [2]:
# Cell 2: Import libraries and setup raw data
import pulp
import pandas as pd
import numpy as np

# 1. Facilities Data (Daily Demand)
facilities_data = {
    'MED_CENTER': {'Type': 'Hospital', 'Daily_Demand': 80},
    'ENG_BUILDING': {'Type': 'Academic', 'Daily_Demand': 30},
    'SCIENCE_HALL': {'Type': 'Academic', 'Daily_Demand': 35},
    'DORM_A': {'Type': 'Residential', 'Daily_Demand': 55},
    'DORM_B': {'Type': 'Residential', 'Daily_Demand': 45},
    'LIBRARY': {'Type': 'Academic', 'Daily_Demand': 25}
}

# 2. Warehouse Data (Daily Capacity & Costs)
warehouses_data = {
    'WH_NORTH': {'Daily_Capacity': 400, 'Construction_Cost': 300000, 'Daily_Op_Cost': 800},
    'WH_SOUTH': {'Daily_Capacity': 350, 'Construction_Cost': 280000, 'Daily_Op_Cost': 700},
    'WH_EAST': {'Daily_Capacity': 450, 'Construction_Cost': 320000, 'Daily_Op_Cost': 900}
}

# Lists for easy iteration
facilities = list(facilities_data.keys())
warehouses = list(warehouses_data.keys())

print("Raw Data Loaded Successfully.")

Raw Data Loaded Successfully.


## "Annualization"
Annual Demand: Daily Demand × 365

Annual Capacity: Daily Capacity × 365

Annual Fixed Cost: (Construction Cost ÷ 10 years) + (Daily Op Cost × 365 days)

In [3]:
# Cell 3: Data Pre-processing (Annualizing)

# 1. Annualize Demand
annual_demand = {}
for f in facilities:
    annual_demand[f] = facilities_data[f]['Daily_Demand'] * 365

# 2. Annualize Capacity
annual_capacity = {}
for w in warehouses:
    annual_capacity[w] = warehouses_data[w]['Daily_Capacity'] * 365

# 3. Calculate Annual Fixed Costs
# Formula: (Construction / 10) + (Daily Ops * 365)
fixed_costs = {}
for w in warehouses:
    construction_part = warehouses_data[w]['Construction_Cost'] / 10
    operational_part = warehouses_data[w]['Daily_Op_Cost'] * 365
    fixed_costs[w] = construction_part + operational_part

print("--- Annualized Data ---")
print("Annual Demands:", annual_demand)
print("Annual Fixed Costs:", fixed_costs)

--- Annualized Data ---
Annual Demands: {'MED_CENTER': 29200, 'ENG_BUILDING': 10950, 'SCIENCE_HALL': 12775, 'DORM_A': 20075, 'DORM_B': 16425, 'LIBRARY': 9125}
Annual Fixed Costs: {'WH_NORTH': 322000.0, 'WH_SOUTH': 283500.0, 'WH_EAST': 360500.0}


## Generate Transportation Costs

In [4]:
# Cell 4: Generate Transportation Costs Matrix
# Note: Using random values within the problem's specified range ($3.68 - $5.03)

np.random.seed(42) # Ensures the numbers stay the same every time we run this

transport_costs = {}
print("--- Transportation Costs (Per Unit) ---")
for w in warehouses:
    for f in facilities:
        # Generate a random cost between 3.68 and 5.03
        cost = np.round(np.random.uniform(3.68, 5.03), 2)
        transport_costs[(w, f)] = cost
        print(f"Cost from {w} to {f}: ${cost}")

--- Transportation Costs (Per Unit) ---
Cost from WH_NORTH to MED_CENTER: $4.19
Cost from WH_NORTH to ENG_BUILDING: $4.96
Cost from WH_NORTH to SCIENCE_HALL: $4.67
Cost from WH_NORTH to DORM_A: $4.49
Cost from WH_NORTH to DORM_B: $3.89
Cost from WH_NORTH to LIBRARY: $3.89
Cost from WH_SOUTH to MED_CENTER: $3.76
Cost from WH_SOUTH to ENG_BUILDING: $4.85
Cost from WH_SOUTH to SCIENCE_HALL: $4.49
Cost from WH_SOUTH to DORM_A: $4.64
Cost from WH_SOUTH to DORM_B: $3.71
Cost from WH_SOUTH to LIBRARY: $4.99
Cost from WH_EAST to MED_CENTER: $4.8
Cost from WH_EAST to ENG_BUILDING: $3.97
Cost from WH_EAST to SCIENCE_HALL: $3.93
Cost from WH_EAST to DORM_A: $3.93
Cost from WH_EAST to DORM_B: $4.09
Cost from WH_EAST to LIBRARY: $4.39


## Initialize Model & Decision Variables

In [5]:
# Cell 5: Initialize Model and Decision Variables

# 1. Create the Problem
prob = pulp.LpProblem("Campus_City_Emergency_Distribution", pulp.LpMinimize)

# 2. Decision Variable 1: Open Warehouse? (Binary: 0 or 1)
warehouse_vars = pulp.LpVariable.dicts("Open", warehouses, cat='Binary')

# 3. Decision Variable 2: Shipment Quantity (Continuous, must be >= 0)
flow_vars = pulp.LpVariable.dicts("Ship", (warehouses, facilities), lowBound=0, cat='Continuous')

print("Model and Variables Initialized.")

Model and Variables Initialized.


## Define Objective Function & Constraints

Objective: We tell the solver to minimize (Fixed Costs of open warehouses + Shipping Costs of moving goods).

Constraint 1 (Demand): Every facility must get exactly what it needs.

Constraint 2 (Capacity): You can't ship more than a warehouse holds. Also, if a warehouse is closed (0), capacity becomes 0.

Constraint 3 (Selection): We force the solver to pick exactly 2 warehouses.

Constraint 4 (Budget): The total money spent must be under $1.5 Million.

In [6]:
# Cell 6: Define Objective and Constraints

# --- OBJECTIVE FUNCTION ---
# Minimize: Sum of (Fixed Costs) + Sum of (Transportation Costs)
prob += (
    pulp.lpSum([fixed_costs[w] * warehouse_vars[w] for w in warehouses]) +
    pulp.lpSum([transport_costs[(w, f)] * flow_vars[w][f] for w in warehouses for f in facilities])
), "Total_Annual_Cost"

# --- CONSTRAINTS ---

# 1. Demand Satisfaction: Each facility must receive its annual demand
for f in facilities:
    prob += pulp.lpSum([flow_vars[w][f] for w in warehouses]) == annual_demand[f], f"Demand_{f}"

# 2. Capacity Constraints: Warehouse output <= Capacity * Open_Status
# If warehouse is closed (0), capacity becomes 0.
for w in warehouses:
    prob += pulp.lpSum([flow_vars[w][f] for f in facilities]) <= annual_capacity[w] * warehouse_vars[w], f"Capacity_{w}"

# 3. Warehouse Selection: Select exactly 2 warehouses
prob += pulp.lpSum([warehouse_vars[w] for w in warehouses]) == 2, "Select_Exactly_Two"

# 4. Budget Constraint: Total Cost <= $1,500,000
total_cost_equation = (
    pulp.lpSum([fixed_costs[w] * warehouse_vars[w] for w in warehouses]) +
    pulp.lpSum([transport_costs[(w, f)] * flow_vars[w][f] for w in warehouses for f in facilities])
)
prob += total_cost_equation <= 1500000, "Budget_Limit"

print("Objective and Constraints Added.")

Objective and Constraints Added.


## Solve the Problem

In [7]:
# Cell 7: Solve the Model
prob.solve()

# Check Status
print("Solution Status:", pulp.LpStatus[prob.status])

Solution Status: Optimal


## Output Report

In [8]:
# Cell 8: Generate Final Report

if pulp.LpStatus[prob.status] == 'Optimal':
    total_cost = pulp.value(prob.objective)

    print("="*60)
    print(f"OPTIMIZATION RESULTS (Micro-Project 1)")
    print("="*60)
    print(f"Total Annual Cost: ${total_cost:,.2f}")
    print(f"Budget Limit:      $1,500,000.00")
    print(f"Under Budget By:   ${1500000 - total_cost:,.2f}")
    print("-" * 60)

    print("\n--- WAREHOUSE DECISIONS ---")
    for w in warehouses:
        status = "OPEN" if warehouse_vars[w].varValue == 1 else "CLOSED"
        if status == "OPEN":
            # Calculate how much capacity is used
            used_capacity = sum(flow_vars[w][f].varValue for f in facilities)
            utilization = (used_capacity / annual_capacity[w]) * 100
            print(f"[{status}] {w}")
            print(f"   - Fixed Cost: ${fixed_costs[w]:,.2f}")
            print(f"   - Utilization: {used_capacity:,.0f} / {annual_capacity[w]:,.0f} units ({utilization:.1f}%)")

    print("\n--- DISTRIBUTION PLAN (Units Shipped) ---")
    print(f"{'From':<15} {'To':<15} {'Quantity':<10} {'Cost'}")
    for w in warehouses:
        if warehouse_vars[w].varValue == 1:
            for f in facilities:
                qty = flow_vars[w][f].varValue
                if qty > 0:
                    cost = qty * transport_costs[(w, f)]
                    print(f"{w:<15} {f:<15} {qty:<10.0f} ${cost:,.2f}")

else:
    print("No optimal solution found. The constraints might be too tight (e.g., budget too low).")

OPTIMIZATION RESULTS (Micro-Project 1)
Total Annual Cost: $1,012,329.00
Budget Limit:      $1,500,000.00
Under Budget By:   $487,671.00
------------------------------------------------------------

--- WAREHOUSE DECISIONS ---
[OPEN] WH_NORTH
   - Fixed Cost: $322,000.00
   - Utilization: 29,200 / 146,000 units (20.0%)
[OPEN] WH_SOUTH
   - Fixed Cost: $283,500.00
   - Utilization: 69,350 / 127,750 units (54.3%)

--- DISTRIBUTION PLAN (Units Shipped) ---
From            To              Quantity   Cost
WH_NORTH        DORM_A          20075      $90,136.75
WH_NORTH        LIBRARY         9125       $35,496.25
WH_SOUTH        MED_CENTER      29200      $109,792.00
WH_SOUTH        ENG_BUILDING    10950      $53,107.50
WH_SOUTH        SCIENCE_HALL    12775      $57,359.75
WH_SOUTH        DORM_B          16425      $60,936.75
